In [1]:
import sys
import os
from datetime import datetime

root_dir = '../../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import optim
import pandas as pd

pd.set_option('display.max_columns', 100)

from modules import losses, models, samplers, searches, evaluators, trainers, datasets, distributions

In [2]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
n_feedback = dataset.n_pos_pairs
train_set, test_set = dataset.get_train_and_test_set(neg_pair_weight=10)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.FloatTensor(test_set).to(device)

In [4]:
# The size of recommendation set (K)
ks = [10]

score_function_dict = {
    "Recall"       : evaluators.recall,
    "Unpopularity" : evaluators.unpopularity,
    "Serendipity"  : evaluators.serendipity,
    "Long-tail rate": evaluators.longtail_rate,
}
userwise = evaluators.UserwiseEvaluator(test_set, score_function_dict, ks)

In [5]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

In [6]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 10
no_progressbar = False

model = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
feedback_num_df = dataset.feedback_num_train_data()
criterion = losses.MinorTripletLoss(feedback_num_df, margin=1, a=1, b=0.5).to(device)
trainer = trainers.BaseTrainer(model, optimizer, criterion, sampler, no_progressbar)

In [7]:
trainer.fit(n_batch, n_epoch)

epoch10 avg_loss:nan: 100%|██████████| 256/256 [00:03<00:00, 72.02it/s]


In [8]:
knn = searches.NearestNeighborhood(model)

In [9]:
trainer.valid(knn, userwise)

100%|██████████| 940/940 [00:03<00:00, 293.91it/s]


In [10]:
trainer.valid_scores

,Recall@10,Unpopularity@10,Serendipity@10,Long-tail rate@10
0,0.136608,312.346732,3.504257,0.761702
